Test for extracting more information from the TRIPS parse

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import requests
import xml.etree.ElementTree as ET
from pyparsing import OneOrMore, nestedExpr
import ast
from diesel import lisp
import re

In [34]:
sentence = "Some women bear children"

params = {
    "input": sentence,
    "extsformat": "svg",
    "tagsformat": "hidden",
    "treecontents": "phrase",
    "treeformat": "LinGO",
    "lfformat": "svg",
    "no-sense-words": "",
    "tag-type": "",
    "senses-only-for-penn-poss": "",
}

a = requests.get("http://trips.ihmc.us/parser/cgi/parse", params=params)
root = ET.fromstring(a.text)
utt = root.find("utt")
terms = utt.find("terms")
lisp_section = terms.find("lisp")
lispcode = lisp_section.text
print lispcode
lispcode = lispcode.replace("\n","")
# Load the predicates using diesel lisp parser
predicates = lisp.extract_predicates(lispcode)


((ONT::SPEECHACT ONT::V973400 ONT::SA_TELL :CONTENT ONT::V973337 :START 0 :END
  24)
 (ONT::F ONT::V973337 (:* ONT::ENDURING W::BEAR) :EXPERIENCER ONT::V973329
  :NEUTRAL ONT::V973343 :TENSE W::PRES :START 0 :END 24)
 (ONT::INDEF-SET ONT::V973329 (:* ONT::FEMALE-PERSON W::WOMAN) :SIZE ONT::SOME
  :START 0 :END 11)
 (ONT::INDEF-SET ONT::V973343 (:* ONT::CHILD W::CHILD) :START 16 :END 24))


In [33]:
# Along with the predicates, return the word match also. For example, in
# A person may raise money, raise should have role ONT::collect. Return
# mapping from ONT::COLLECT to W::RAISE also
pattern = re.compile("\([:][*] ONT::([A-Za-z\-]*) W::([A-Za-z\-]*)\)([A-Za-z0-9: ]*):START (\d+) :END (\d+)")
role_dict = {}
for match in pattern.finditer(lispcode):
    start = int(match.group(4))
    end = int(match.group(5))
    role_dict[match.group(1).lower()] = match.group(2).lower(), sentence[start:end]
print predicates, role_dict

['(enduring :experiencer female-person :neutral child)', '(sa_tell :content enduring)'] {'female-person': ('woman', 'Some women '), 'enduring': ('bear', 'Some women bear children'), 'child': ('child', 'children')}
